In [1]:
# !pip install -U "git+https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

!git clone -b alex_dev "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 120 (delta 67), reused 86 (delta 39), pack-reused 0
Receiving objects: 100% (120/120), 29.89 KiB | 3.74 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torch.utils.data as data
import random
import numpy as np
from torchsummary import summary

import copy
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
ROOT = '.data'
train_data = datasets.CIFAR10(root = ROOT, 
                              train = True, 
                              download = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data


In [6]:
# Compute means and standard deviations along the R,G,B channel

means = train_data.data.mean(axis = (0,1,2)) / 255
stds = train_data.data.std(axis = (0,1,2)) / 255

In [7]:
from src.transforms import make_transforms

train_transforms, test_transforms = make_transforms(means, stds)

In [8]:
train_data = datasets.CIFAR10(ROOT, 
                              train = True, 
                              download = True, 
                              transform = train_transforms)

test_data = datasets.CIFAR10(ROOT, 
                             train = False, 
                             download = True, 
                             transform = test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

In [10]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [11]:
BATCH_SIZE = 512

train_iterator = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

valid_iterator = data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)

test_iterator = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

**Define our Model

In [12]:
from src.model import ResNet, StemConfig
from src.utils import initialize_parameters, epoch_time

model_architecture = (
    (1, 128, 0.5),
    (2, 128, 0.5),
    (2, 128, 0.25),
    (2, 128, 0.25),
    (2, 196, 0.1),
    (2, 196, 0.1),
)

stem_config = StemConfig(num_channels=128, kernel_size=5, stride=1, padding=2)
model = ResNet(model_architecture, stem_config=stem_config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
from pathlib import Path

path = "/content/drive/MyDrive/School/Tandon MSCS/Classes/CS-GY 6953: Deep Learning/midterm/"
file_path = path + "resnet_alex_48m_dropout.pt"

model_file = Path(file_path)

Need to run a dummy set of data to initialize the lazy modules before we can use torchsummary

In [13]:
if model_file.exists() and model_file.is_file():
  # load our previously trained model
  model.load_state_dict(torch.load(model_file))
else:
  # intialize a new model
  inputs = torch.empty((512, 3, 32, 32))
  inputs.normal_()
  model = model.to(device)
  y = model(inputs.to(device))
  print(y.size())

  model.apply(initialize_parameters)

torch.Size([512, 10])

In [15]:
print(f"num params: {sum([p.numel() for p in model.parameters() if p.requires_grad]):,}")

num params: 4,845,850


In [16]:
from src.engine import train_one_epoch, evaluate

best_loss = float('inf')
EPOCHS  = 20
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
for epoch in range(1, EPOCHS+1):
    start = time.time()

    print(f"Epoch {epoch}")
    train_loss, train_acc = train_one_epoch(model, train_iterator, criterion, optimizer, device)
    train_mins, train_secs = epoch_time(start, time.time())

    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    start = time.time()
    val_loss, val_acc = evaluate(model, valid_iterator, criterion, device)
    val_mins, val_secs = epoch_time(start, time.time())

    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

    if val_loss < best_loss:
        best_loss = val_loss
        # torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/midterm/resnet_alex_485.pt")
        torch.save(model.state_dict(), file_path)

Epoch 1
	Train elapsed: 1:6, loss: 2.1851, acc: 25.56%
	Validation elapsed: 0:2, loss: 1.7608, acc: 36.40%
Epoch 2
	Train elapsed: 1:6, loss: 1.6050, acc: 40.50%
	Validation elapsed: 0:2, loss: 1.4941, acc: 46.13%
Epoch 3
	Train elapsed: 1:5, loss: 1.4197, acc: 48.12%
	Validation elapsed: 0:2, loss: 1.3127, acc: 53.80%
Epoch 4
	Train elapsed: 1:5, loss: 1.2813, acc: 53.17%
	Validation elapsed: 0:2, loss: 1.2289, acc: 56.28%
Epoch 5
	Train elapsed: 1:5, loss: 1.1656, acc: 57.84%
	Validation elapsed: 0:2, loss: 1.1399, acc: 60.38%
Epoch 6
	Train elapsed: 1:5, loss: 1.0602, acc: 61.91%
	Validation elapsed: 0:2, loss: 1.0391, acc: 64.32%
Epoch 7
	Train elapsed: 1:5, loss: 0.9807, acc: 64.85%
	Validation elapsed: 0:2, loss: 0.9122, acc: 68.52%
Epoch 8
	Train elapsed: 1:5, loss: 0.9087, acc: 67.63%
	Validation elapsed: 0:2, loss: 0.8746, acc: 69.34%
Epoch 9
	Train elapsed: 1:5, loss: 0.8516, acc: 69.84%
	Validation elapsed: 0:2, loss: 0.7989, acc: 72.16%
Epoch 10
	Train elapsed: 1:5, loss: 0

## Evaluate the Model  

In [18]:
# model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/midterm/resnet_alex_485.pt"))
model.load_state_dict(torch.load(file_path))
test_loss, test_acc = evaluate(model.to(device), test_iterator, criterion, device)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

Test Loss: 0.5846
Test Accuracy: 81.37%
